# Effect of Unscripted data with pre-trained Scripted data
In this experiment, a dataset of unscripted data for english speakers and PHQ-8 will be created. From this, a Transfer Learning model will be set with the scripted model to see how it can behave. Also, a non new model will be set for this data. 

Importing all the facilities.

In [1]:
from sklearn.model_selection import KFold
from data_Loader import dataLoaderPickle, justDatasetLoaderPickle, dataCreator, dataSaver, dataMLTCreator
from networks_v2 import AttentionModel, AnxietyFromDepression, HydraNetTL, Scripted2Unscripted
from optimizer import train2, test2, optimizerNet, test, train, testMLT, trainMLT
from optimizer import IterMeter

import torch
import torch.nn as nn
from comet_ml import Experiment

import os
import numpy as np

from torch.utils.data import DataLoader, random_split, SubsetRandomSampler

c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Creating dataset

In [2]:
train_loader = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\Raw_Data_v2\\RADAR-MDD-KCL-s1\\RADAR-MDD-KCL-s1"
test_loader = ""

name =[]
data = []
audio = []

name = [x[0] for x in os.walk(train_loader)]

name.pop(0)

'C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\Raw_Data_v2\\RADAR-MDD-KCL-s1\\RADAR-MDD-KCL-s1'

In [4]:
dataset = dataMLTCreator(train_loader, pickleName="Unscripted_Eng_PHQ8_MLT", task="unscripted")

Signal processed 1_1
New Spectrogram
The new spectrogram has a target
(2, 500, 40) Spectrogram shape
New Spectrogram augmented
The new spectrogram has a target
(2, 500, 40) Spectrogram shape NOISE
DATA AUGMENTED
Length Spectrograms:  (5, 500, 40)
Length labels depresion:  4
Length labels anxiety:  4
Signal processed 2_1
New Spectrogram
The new spectrogram has a target
(1, 500, 40) Spectrogram shape
New Spectrogram augmented
The new spectrogram has a target
(1, 500, 40) Spectrogram shape NOISE
DATA AUGMENTED
Length Spectrograms:  (7, 500, 40)
Length labels depresion:  6
Length labels anxiety:  6
Signal processed 3_1
New Spectrogram
The new spectrogram has a target
(2, 500, 40) Spectrogram shape
New Spectrogram augmented
The new spectrogram has a target
(2, 500, 40) Spectrogram shape NOISE
DATA AUGMENTED
Length Spectrograms:  (11, 500, 40)
Length labels depresion:  10
Length labels anxiety:  10
Signal processed 4_1
New Spectrogram
The new spectrogram has a target
(1, 500, 40) Spectrogram

In [3]:
dataSaver(dataset, "Unscripted_Eng_PHQ8")

NameError: name 'dataset' is not defined

In [2]:
dataset_uns = justDatasetLoaderPickle('Unscripted_Eng_PHQ8_MLT')

In [5]:
tar = []
import matplotlib.pyplot as plt
for x, y,z in dataset_uns:
    tar.append(y)

n, bins, patches = plt.hist(x=tar,bins='auto',color='#0504aa',alpha=0.7, rwidth=0.85)
plt.grid(axis='y',alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Is my data balanced?')
plt.text(23,45,r'$\mu15, b=3$')
maxfreq = n.max()
plt.ylim(ymax=np.ceil(maxfreq/10)*10 if maxfreq % 10 else maxfreq + 10)
plt.show()

## Loading the model 

In [3]:
#Constants
learning_Rate = 0.0005
batch_size = 128
epochs = 200
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 5,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [4]:
filename = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\model_PHQ8_Scripted_Eng.pt"
infile = open(filename, 'rb')
model_pretrained = torch.load(infile)
infile.close()

In [4]:
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

#Optimizing Model
weights = [0.75, 0.75, 0.75, 0.8, 1.0]
class_weights = torch.FloatTensor(weights)
criterion_dep = nn.CrossEntropyLoss(weight=class_weights).to(device)
criterion_anx = nn.CrossEntropyLoss().to(device)
iter_meter = IterMeter()

In [5]:
from sklearn.metrics import confusion_matrix
import seaborn as sn 
import pandas as pd
import torch.nn.functional as F 
import matplotlib.pyplot as plt

def trainXD(epochs, model_mlt, train_loader, optimizer, criterion, iter_meter):
    acc_dep_list = []
    loss_list = []
    label_dep_list = []
    predicted_list_dep = []
    loss_tot_list = []
    accuracy_dep = []

    #Training
    for epoch in range(epochs):
        model_mlt.train()
        total_training_loss = 0

        for x,y,z in train_loader:
            inputs = x
            label_dep = y.long()
            label_dep_list.extend(label_dep.detach().numpy())

            optimizer.zero_grad()
            phq8 = model_mlt(inputs.float())
            phq8 = phq8.squeeze(0)

            predicted_list_dep.extend((torch.max(torch.exp(F.log_softmax(phq8,dim=1)),1)[1]).detach().numpy())

            loss = criterion_dep(phq8,label_dep)

            loss.backward()
            optimizer.step()
            iter_meter.step()
            total_training_loss += loss

            loss_list.append(loss.item())
            #Track accuracy
            total_dep = label_dep.size(0)
            _, predicted = torch.max(phq8.data,1)
            correct_dep = (predicted == label_dep).sum().item()
            acc_dep_list.append(correct_dep/total_dep)  

        print('Train Epoch: {} \tLoss: {:.4f}\tDepression Accuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_dep_list)
        ))      
        loss_tot_list.append(np.mean(loss_list))
        accuracy_dep.append(np.mean(acc_dep_list))
        
    #Printing Confusion Matrix
    classes_dep = ('0','1','2','3','4')
    cf_matrix_dep = confusion_matrix(label_dep_list, predicted_list_dep)
    df_cm = pd.DataFrame(cf_matrix_dep/np.sum(cf_matrix_dep)*10,index = [i for i in classes_dep], columns=[i for i in classes_dep])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Depression Training')

    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Training Loss')
    axs[0].set(xlabel= 'Epoch', ylabel='Loss')
    axs[1].plot(range(epochs), accuracy_dep)
    axs[1].set_title('Training Depression Accuracy')
    axs[1].set(xlabel= 'Epoch', ylabel='Accuracy')
    plt.show()

In [6]:
def testXD(epochs, model_mlt, train_loader, optimizer, criterion, iter_meter):
    acc_dep_list = []
    loss_list = []
    label_dep_list = []
    predicted_list_dep = []
    loss_tot_list = []
    accuracy_dep = []

    #Training
    for epoch in range(epochs):
        model_mlt.train()
        total_training_loss = 0

        for x,y,z in train_loader:
            inputs = x
            label_dep = y.long()
            label_dep_list.extend(label_dep.detach().numpy())

            #optimizer.zero_grad()
            phq8 = model_mlt(inputs.float())
            phq8 = phq8.squeeze(0)

            predicted_list_dep.extend((torch.max(torch.exp(F.log_softmax(phq8,dim=1)),1)[1]).detach().numpy())

            loss = criterion_dep(phq8,label_dep)

            #loss.backward()
            #optimizer.step()
            iter_meter.step()
            total_training_loss += loss

            loss_list.append(loss.item())
            #Track accuracy
            total_dep = label_dep.size(0)
            _, predicted = torch.max(phq8.data,1)
            correct_dep = (predicted == label_dep).sum().item()
            acc_dep_list.append(correct_dep/total_dep)  

        print('Test Epoch: {} \tLoss: {:.4f}\tDepression Accuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_dep_list)
        ))      
        loss_tot_list.append(np.mean(loss_list))
        accuracy_dep.append(np.mean(acc_dep_list))
        
    #Printing Confusion Matrix
    classes_dep = ('0','1','2','3','4')
    cf_matrix_dep = confusion_matrix(label_dep_list, predicted_list_dep)
    df_cm = pd.DataFrame(cf_matrix_dep/np.sum(cf_matrix_dep)*10,index = [i for i in classes_dep], columns=[i for i in classes_dep])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Depression Testing')

    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Test Loss')
    axs[0].set(xlabel= 'Epoch', ylabel='Loss')
    axs[1].plot(range(epochs), accuracy_dep)
    axs[1].set_title('Test Depression Accuracy')
    axs[1].set(xlabel= 'Epoch', ylabel='Accuracy')
    plt.show()

In [10]:
epochs = 150

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_uns)))):
    print('Fold {}'.format(fold+1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loaderUns = DataLoader(dataset_uns, batch_size=batch_size, sampler=train_sampler)
    test_loaderUns = DataLoader(dataset_uns, batch_size=batch_size, sampler=test_sampler)

    model_TF = AttentionModel(
        hparams['n_cnn_layers'],
        hparams['rnn_dim'],
        hparams['h_rnn_layers'],
        hparams['n_rnn_layers'],
        hparams['n_class'],
        hparams['stride'],
        hparams['dropout']
    ).to(device).float()

    optimizer, scheduler = optimizerNet(model_TF, hparams)

    trainXD(epochs, model_TF, train_loaderUns, optimizer, criterion_dep, iter_meter)
    testXD(epochs, model_TF, test_loaderUns, optimizer, criterion_anx, iter_meter)

Fold 1
Train Epoch: 0 	Loss: 1.5285	Depression Accuracy: 0.3161
Train Epoch: 1 	Loss: 1.4947	Depression Accuracy: 0.3264
Train Epoch: 2 	Loss: 1.4751	Depression Accuracy: 0.3322
Train Epoch: 3 	Loss: 1.4618	Depression Accuracy: 0.3368
Train Epoch: 4 	Loss: 1.4515	Depression Accuracy: 0.3410
Train Epoch: 5 	Loss: 1.4435	Depression Accuracy: 0.3439
Train Epoch: 6 	Loss: 1.4366	Depression Accuracy: 0.3464
Train Epoch: 7 	Loss: 1.4309	Depression Accuracy: 0.3486
Train Epoch: 8 	Loss: 1.4258	Depression Accuracy: 0.3504
Train Epoch: 9 	Loss: 1.4212	Depression Accuracy: 0.3524
Train Epoch: 10 	Loss: 1.4172	Depression Accuracy: 0.3541
Train Epoch: 11 	Loss: 1.4134	Depression Accuracy: 0.3558
Train Epoch: 12 	Loss: 1.4100	Depression Accuracy: 0.3576
Train Epoch: 13 	Loss: 1.4068	Depression Accuracy: 0.3592
Train Epoch: 14 	Loss: 1.4039	Depression Accuracy: 0.3603
Train Epoch: 15 	Loss: 1.4011	Depression Accuracy: 0.3615
Train Epoch: 16 	Loss: 1.3984	Depression Accuracy: 0.3630
Train Epoch: 17 	

KeyboardInterrupt: 